# Flowtuple analysis demo for AIMS 2025

**Author(s):** M. Gao (magao@ucsd.edu) Ricky Mok (cskpmok@caida.org)

## 0 - Imports

In [43]:
# Import Libraries
import pyspark
from pyspark import SQLContext
from pyspark import SparkContext
import pyspark.sql.functions as psf
import pyspark.sql.types as pst
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number
from pyspark.sql import SparkSession

import os
from datetime import datetime, timedelta, date
import dateutil.relativedelta as relativedelta
import IPython
import pandas as pd
import numpy as np
import warnings

# Find Spark
import findspark
findspark.init()

# PySpark imports
import pyspark
from pyspark import SQLContext
from pyspark import SparkContext
from pyspark import StorageLevel
import pyspark.sql.functions as F
import pyspark.sql.types as pst
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number, lit

In [2]:
print(os.environ["SPARK_HOME"])

/scratch/magao/job_42756715/spark-3.2.1


## 1 - Spark Configuration

In [3]:
user=%env USER
user

'magao'

In [4]:
hostname = !hostname --fqdn
hostname[0]

'exp-2-28.expanse.sdsc.edu'

In [5]:
PROJECT = "csd939"

In [6]:
SLURM_JOB_ID = !squeue -u $LOGNAME | grep $HOSTNAME | awk '{print $1}'
SLURM_JOB_ID = SLURM_JOB_ID[0] if SLURM_JOB_ID else None  # pick first match or None
print(SLURM_JOB_ID)

42756715


In [7]:
# os.environ["PYSPARK_PYTHON"] = "/opt/anaconda3/envs/spark3/bin/python"
# os.environ["SPARK_HOME"] = "/usr/local/spark-3.1.2-bin-hadoop3.2"
os.environ["HADOOP_OPTS"] = "-Djava.library.path=/cm/shared/apps/spack/cpu/opt/spack/linux-centos8-zen/gcc-8.3.1/hadoop/3.2.2/lib/native"
os.environ['PYSPARK_SUBMIT_ARGS'] = fr"--jars /scratch/{user}/job_{SLURM_JOB_ID}/spark-3.2.1/ext_jars/* --packages org.apache.spark:spark-avro_2.12:3.2.1 pyspark-shell"
os.environ["SPARK_LIB"] = os.environ["SPARK_HOME"]

In [8]:
# Set your swift credentials
UCSD_NT_S3_ACCESS_KEY = %env UCSD_NT_S3_ACCESS_KEY
UCSD_NT_S3_SECRET_KEY = %env UCSD_NT_S3_SECRET_KEY

In [9]:
# Set spark configurations to align with the slrum configuration
SPARK_CORES_MAX = "128"
SPARK_EXEC_CPU  = "122"
SPARK_EXEC_MEM  = "250G"

In [10]:
# MASTER_URL=fr"spark://{hostname[0]}:7077"
MASTER_URL=fr"local[*]"

spark = SparkSession.builder.master(MASTER_URL).appName('spark-cluster'
).config("fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider"
).config("fs.s3a.access.key", UCSD_NT_S3_ACCESS_KEY
).config("fs.s3a.secret.key", UCSD_NT_S3_SECRET_KEY
).config("fs.s3a.endpoint", "https://hermes.caida.org").config("fs.s3a.path.style.access", "true"
).config("fs.s3a.block.size", "64M").config("fs.s3a.readahead.range", "128K"
).config("fs.s3a.experimental.input.fadvise", "sequential"
).config("fs.s3a.connection.maximum", 256
).config("spark.driver.extraJavaOptions", f"-Djava.io.tmpdir=/scratch/{user}/job_{SLURM_JOB_ID}/spark_scratch"
).config("spark.executor.extraJavaOptions", f"-Djava.io.tmpdir=/scratch/{user}/job_{SLURM_JOB_ID}/spark_scratch"         
).config("spark.cores.max", SPARK_CORES_MAX
).config("spark.driver.cores","6"
).config("spark.driver.memory","6G"
).config("spark.executor.cores", SPARK_EXEC_CPU
).config("spark.executor.memory", SPARK_EXEC_MEM
# ).config("spark.executor.memoryOverhead", "4G"
).config("io.file.buffer.size", "67108864"
).config("spark.submit.deploymode", "client"
).config("spark.buffer.size", "67108864"
).config("spark.network.timeout", "300s"
).config("spark.sql.session.timeZone", "UTC"
).config("spark.sql.files.ignoreCorruptFiles", "true" # Set this when analyzing periods of high traffic volatility, during which there may be corrupt files
).getOrCreate()

print(spark)

:: loading settings :: url = jar:file:/scratch/magao/job_42756715/spark-3.2.1/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/magao/.ivy2/cache
The jars for the packages stored in: /home/magao/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0fa8872a-4e38-485e-ab68-aef2a9989ea5;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.2.1 in central
	found org.tukaani#xz;1.8 in central
	found org.spark-project.spark#unused;1.0.0 in central
:: resolution report :: resolve 266ms :: artifacts dl 5ms
	:: modules in use:
	org.apache.spark#spark-avro_2.12;3.2.1 from central in [default]
	org.spark-project.spark#unused;1.0.0 from central in [default]
	org.tukaani#xz;1.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |  

In [11]:
# sc = SparkContext(conf=spark)
# sc.setLogLevel('ERROR')
sqlcontext = SQLContext(spark)

/scratch/magao/job_42756715/.venv/lib64/python3.9/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [12]:
spark

In [13]:
spark.conf.get("spark.executor.cores")

'122'

## 2 - Functions

In [14]:
import socket, struct
import ipaddress

def long2ip(long):
    return ipaddress.ip_network(socket.inet_ntoa(struct.pack('!L', long)) + '/255.255.255.255', strict=False)

def long2ip24subnet(long):
    return ipaddress.ip_network(socket.inet_ntoa(struct.pack('!L', long)) + '/255.255.255.0', strict=False)

def long2ip16subnet(long):
    return ipaddress.ip_network(socket.inet_ntoa(struct.pack('!L', long)) + '/255.255.0.0', strict=False)

'''
Convert an IP string to long
'''
def ip2long(ip):
    packedIP = socket.inet_aton(ip)
    return struct.unpack("!L", packedIP)[0]

In [15]:
'''
Input: List of IPv4 Subnets in CIDR Notation e.g. ('0.0.0.0/16')

Returns the set of all IP addresses as integers within the subnet.
'''
def explode_prefix(prefix_list):
    prefix_set = set()
    
    for pfx in prefix_list:
        for ip in ipaddress.IPv4Network(pfx):
            prefix_set.add(ip2long(str(ip)))
            
    return prefix_set

In [16]:
import pytz

def load_ft_pyspark(avro_files):
    df = sqlcontext.read.format('avro').load(avro_files)
    df = df.withColumn('time', psf.to_utc_timestamp((df.time).cast(dataType=pst.TimestampType()), 'UTC'))
    # df = df.withColumn('time', psf.from_unixtime(df.time))
    return df

## 3 - Analysis Configuration

In [17]:
### Papermill Parameters ###

OUT_DIR = f'/expanse/lustre/projects/{PROJECT}/{user}/shared_data/case_study_1'
START_1 = '2024-01-01 00:00'
END_1 = '2024-01-01 00:30'

## 4 - Load .avro Files

### 4.1 - Convert Timestamps to DateTime

In [18]:
# Inclusive start and end times
start_1 = pd.to_datetime(START_1)
end_1 = pd.to_datetime(END_1)

### 4.2 - Load Files

In [19]:
def generate_avro_uris(avro_filename, start_dt, end_dt, s3_pre_uri='s3a://telescope-ucsdnt-avro-flowtuple-v4-2024'):
    avro_df = pd.read_parquet(avro_filename)
    avro_df['datetime'] = pd.to_datetime(avro_df['datetime'])
    avro_df = avro_df.set_index('datetime')
    
    ### Select .avro's within timeframe
    selected_avros = avro_df[(avro_df.index >= start_dt) & (avro_df.index <= end_dt)]
    print(f'Avro filecount: {len(selected_avros)}')
    
    ### Build URI's
    selected_avro_uris = selected_avros.filename.apply(lambda x: f'{s3_pre_uri}/{x}').values
    print(f'Avro uri count: {len(selected_avro_uris)}')
    
    return selected_avro_uris

Use per-computed list of available flowtuple files. 

In [20]:
avro_filename = '../ft4_file_lists/ft4_2024_files.parquet.gzip'
s3_pre_uri = 's3a://telescope-ucsdnt-avro-flowtuple-v4-2024/'

avro_uris = generate_avro_uris(avro_filename, start_1, end_1, s3_pre_uri)

Avro filecount: 7
Avro uri count: 7


In [21]:
%%time

spark_df = load_ft_pyspark(list(avro_uris))

25/09/26 02:08:45 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


CPU times: user 6.02 ms, sys: 982 µs, total: 7.01 ms
Wall time: 4.6 s


## 5 - Optional Filters

#### Source Filters

In [22]:
src_ip_filter = set([
    # '137.110.41.27',
    # '137.110.41.248',
    # '137.110.33.50',
    # '137.110.60.41',
])

src_subnet_filter = set([
    # '137.110.0.0/16'
])

src_asn_filter = [
]

# Convert to integers
src_ip_filter = set(map(ip2long, src_ip_filter))
src_subnet_filter = explode_prefix(src_subnet_filter)
# Union
ip_filter = src_ip_filter.union(src_subnet_filter)

print(f'IP filter cardinality: {len(ip_filter)}')
print(f'ASN filter cardinality: {len(src_asn_filter)}')

IP filter cardinality: 0
ASN filter cardinality: 0


#### Dest. Filters

In [23]:
dst_ip_filter = set([
])

dst_port_filter = set([
    # 25, 1723, 2379, 3306, 4567, 5060, 6443, 7170, 7547, 8008, 8009, 8085, 8089, 9200,
    # 10250, 30005, 50001
])

print(f'DST_IP filter cardinality: {len(dst_ip_filter)}')
print(f'DST_PORT filter cardinality: {len(dst_port_filter)}')

DST_IP filter cardinality: 0
DST_PORT filter cardinality: 0


#### Misc. Filters

In [24]:
proto_filter = set([
    17 # UDP
])

print(f'Protocol filter cardinality: {len(proto_filter)}')

Protocol filter cardinality: 1


### 5.1 - Apply Filters

In [25]:
def create_filter_df(items, col_name):
    return spark.createDataFrame(
        [(item,) for item in items],
        [col_name]
    )

def apply_filter_df(orig_df, filter_df, col_name):
    import pyspark.sql.functions as F
    return orig_df.join(
            F.broadcast(
                filter_df
            ),
            on=col_name
    )

In [26]:
traff_df = spark_df

if len(ip_filter) > 0:
    print(f'Applying Source IP Filter')
    ip_filter_df = create_filter_df(ip_filter, 'src_ip')
    traff_df = traff_df.join(
        F.broadcast(
            ip_filter_df
            ),
        on='src_ip'
    )

if len(src_asn_filter) > 0:
    print(f'Applying Source ASN Filter')
    asn_filter_df = create_filter_df(src_asn_filter, 'prefix2asn')
    traff_df = traff_df.join(
        F.broadcast(
            asn_filter_df
            ),
        on='prefix2asn'
    )
    
if len(dst_port_filter) > 0:
    print(f'Applying Dest. Port Filter')
    dp_filter_df = create_filter_df(dst_port_filter, 'dst_port')
    traff_df = traff_df.join(
        F.broadcast(
            dp_filter_df
            ),
        on='dst_port'
    )

if len(proto_filter) > 0:
    print(f'Applying Protocol Filter')
    proto_filter_df = create_filter_df(proto_filter, 'protocol')
    traff_df = traff_df.join(
        F.broadcast(
            proto_filter_df
            ),
        on='protocol'
    )

Applying Protocol Filter


## 6 - Traffic Metric Analysis

### Functions

In [27]:
def list_len(list):
    if list is None:
        return 0
    else:
        return len(list)

### Packet Counts + Unique Source IP Counts per Subnet

In [45]:
%%time

spark.sparkContext.setLogLevel("ERROR")
result = spark_df.groupby('dst_net').agg(
    F.sum('packet_cnt'),
    F.countDistinct('src_ip')
).toPandas()
spark.sparkContext.setLogLevel("WARN")

[Stage 26:===============>                                     (57 + 128) / 200]

CPU times: user 33.6 ms, sys: 5.88 ms, total: 39.5 ms
Wall time: 12.8 s


In [29]:
result

,dst_net,sum(packet_cnt),count(src_ip)
0,740687872,14080642,75170
1,750452736,12827811,71780
2,738459648,11641835,72907
3,739573760,13799298,78457
4,743243776,12783266,72369
...,...,...,...
166,744030208,12606027,72776
167,739704832,13952883,73179
168,738656256,12711888,71954
169,743964672,12729760,73020


In [30]:
result['dst_net'] = result['dst_net'].apply(lambda ip_int: long2ip(ip_int)).apply(lambda ip: str(ip).split('.')[1])

In [31]:
result['dst_net'] = result['dst_net'].astype(int)

In [32]:
result['pc_percentile_rank'] = result['sum(packet_cnt)'].rank(pct=True)
result['src_ip_percentile_rank'] = result['count(src_ip)'].rank(pct=True)

In [33]:
result.sort_values(by='sum(packet_cnt)', ascending=True)

,dst_net,sum(packet_cnt),count(src_ip),pc_percentile_rank,src_ip_percentile_rank
19,31,241700,12755,0.005848,0.005848
161,134,2526131,42938,0.011696,0.011696
121,3,6454349,60133,0.017544,0.023392
55,132,6821887,59275,0.023392,0.017544
22,154,9232399,64882,0.029240,0.029240
...,...,...,...,...,...
52,177,16515360,73506,0.976608,0.754386
103,129,16705809,72614,0.982456,0.426901
67,156,16805715,74804,0.988304,0.912281
104,178,17280556,72100,0.994152,0.222222


In [34]:
result.sort_values(by='pc_percentile_rank')

,dst_net,sum(packet_cnt),count(src_ip),pc_percentile_rank,src_ip_percentile_rank
19,31,241700,12755,0.005848,0.005848
161,134,2526131,42938,0.011696,0.011696
121,3,6454349,60133,0.017544,0.023392
55,132,6821887,59275,0.023392,0.017544
22,154,9232399,64882,0.029240,0.029240
...,...,...,...,...,...
52,177,16515360,73506,0.976608,0.754386
103,129,16705809,72614,0.982456,0.426901
67,156,16805715,74804,0.988304,0.912281
104,178,17280556,72100,0.994152,0.222222


In [35]:
result.sort_values(by='count(src_ip)')

,dst_net,sum(packet_cnt),count(src_ip),pc_percentile_rank,src_ip_percentile_rank
19,31,241700,12755,0.005848,0.005848
161,134,2526131,42938,0.011696,0.011696
55,132,6821887,59275,0.023392,0.017544
121,3,6454349,60133,0.017544,0.023392
22,154,9232399,64882,0.029240,0.029240
...,...,...,...,...,...
12,99,13434613,77479,0.637427,0.976608
94,0,12670361,77970,0.327485,0.982456
3,21,13799298,78457,0.801170,0.988304
123,190,13635835,95298,0.719298,0.994152


In [40]:
%%time

resultcountry = spark_df.groupby('netacq_country').agg(
    F.sum('packet_cnt'), 
).toPandas()

[Stage 12:=====================================================>(126 + 2) / 128]

CPU times: user 22.1 ms, sys: 4.81 ms, total: 27 ms
Wall time: 11.2 s


In [37]:
resultcountry

,netacq_country,sum(packet_cnt)
0,MM,9129
1,DZ,647499
2,LT,13454109
3,CI,38362
4,FI,553478
...,...,...
209,LI,31
210,GM,21
211,GI,5
212,YT,4


In [41]:
%%time

resultasn= spark_df.groupby('prefix2asn').agg(
    F.sum('packet_cnt'), 
).toPandas()

CPU times: user 75.7 ms, sys: 9.83 ms, total: 85.6 ms
Wall time: 8.81 s


In [42]:
resultasn

,prefix2asn,sum(packet_cnt)
0,8075,1008426
1,56309,4236
2,202561,87
3,150285,798
4,27651,48921
...,...,...
12446,40425,1
12447,31960,25
12448,3999,1
12449,3313,2
